In [2]:
import cv2

from ultralytics import YOLO, solutions
model = YOLO("yolov8m.pt")


In [1]:
results = model("DOGS.jpeg")

# Display the results
results[0].show()

NameError: name 'model' is not defined

In [2]:
import cv2
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector
from your_module import solutions  # Ensure the DistanceCalculation class is correctly imported
from ultralytics import YOLO  # Ensure YOLO model is imported

# Load YOLO model
model = YOLO("yolov8m.pt")  # Adjust to your model's path

# Initialize video capture and object detection
cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error opening camera"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize video writer (optional, if you want to save the video)
video_writer = cv2.VideoWriter("distance_calculation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize distance-calculation object
names = model.model.names
dist_obj = solutions.DistanceCalculation(names=names, view_img=True)

# Initialize FaceMeshDetector
face_detector = FaceMeshDetector(maxFaces=1)

while True:
    success, im0 = cap.read()
    if not success:
        print("Failed to grab frame or camera feed has ended.")
        break

    # Perform object tracking with YOLO
    tracks = model.track(im0, persist=True, show=False)
    im0 = dist_obj.start_process(im0, tracks)
    
    # Perform face mesh detection
    img, faces = face_detector.findFaceMesh(im0, draw=False)
    
    if faces:
        face = faces[0]
        pointLeft = face[145]
        pointRight = face[374]
        w, _ = face_detector.findDistance(pointLeft, pointRight)
        W = 6.3  # Width of face model in cm
        
        # Calculate distance
        f = 840  # Focal length (example value)
        d = (W * f) / w
        
        # Display distance on frame
        cv2.putText(im0, f'Depth: {int(d)}cm', 
                    (face[10][0] - 100, face[10][1] - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow("Camera Feed", im0)
    
    # Optional: Write the frame to video file
    video_writer.write(im0)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'